In [1]:
# Import librerias 
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Rutas de archivos 

# Ruta donde se encuentran los archivos CSV
ruta_datos_mimic = "C:/Users/BiDAlab/TFG_Alfonso/BBDD/mimic-iv-3.1/mimic-iv-3.1/" 

# Importar archivos CSV específicos
admissions = pd.read_csv(ruta_datos_mimic + "hosp/admissions.csv/admissions.csv")
diagnoses = pd.read_csv(ruta_datos_mimic + "hosp/diagnoses_icd.csv/diagnoses_icd.csv")
icu_stays = pd.read_csv(ruta_datos_mimic + "icu/icustays.csv/icustays.csv")

# Ruta guardado
ruta_guardado = "C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo2/MIMIC/CSV/V4/"

In [3]:
# Items id de los antibioticos administrados

itemids_interes = [225837, 225838, 225840, 225842, 225843, 225844, 225845, 225847, 225848, 225850, 225851, 225853, 225855, 225857, 225859, 225860, 225862, 225863, 
                   225865, 225866, 225868, 225869, 225871, 225873, 225875, 225876, 225877, 225879, 225881, 225882, 225883, 225884, 225885, 225886, 225888, 225889,
                   225890, 225892, 225893, 225895, 225896, 225897, 225898, 225899, 225900, 225902, 225903, 225905 ]  # IDs de ejemplo
chunksize = 1000000

filtered_data = []

for chunk in pd.read_csv(ruta_datos_mimic + "icu/inputevents.csv/inputevents.csv", chunksize=chunksize):
    chunk_filtered = chunk[chunk['itemid'].isin(itemids_interes)]
    filtered_data.append(chunk_filtered)

chartevents = pd.concat(filtered_data)

In [4]:
# Guardamos los datos extraidos de chartevents

chartevents.to_csv(ruta_guardado +"chartevents_v4.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_v4.csv'")

Resultados guardados en 'CSV/chartevents_v4.csv'


In [5]:
# Filtramos los pacientes con sepsis con los códigos ICD

# ICD9 : 99591 -> sepsis
#        99592 -> sepsis severa
#        78552 -> septic shock
# ICD10 : A419 -> sepsis no especificada (sangre infectada confirmada pero causa sin especificar)
#         R6521 -> sepsis severa
#         R6520 -> septic shock
codigos_sepsis = ['99591', '99592', '78552', 'R6521', 'A419', 'R6520']

# Filtrar diagnósticos por códigos de sepsis
sepsis_cases = diagnoses[diagnoses['icd_code'].isin(codigos_sepsis)]

# Combinar diagnósticos de sepsis con admisiones para obtener detalles
sepsis_details = pd.merge(sepsis_cases, admissions, on='subject_id', how='inner')

# Mostrar los casos de sepsis
print("Número de casos de sepsis:", sepsis_details['subject_id'].nunique())
print(sepsis_details.head())

Número de casos de sepsis: 15880
   subject_id  hadm_id_x  seq_num icd_code  icd_version  hadm_id_y  \
0    10000826   21086876        2    99591            9   20032235   
1    10000826   21086876        2    99591            9   21086876   
2    10000826   21086876        2    99591            9   28289260   
3    10001401   27012892        3    R6520           10   20144849   
4    10001401   27012892        3    R6520           10   21544441   

             admittime            dischtime deathtime  \
0  2146-12-05 19:07:00  2146-12-12 16:30:00       NaN   
1  2146-12-18 17:39:00  2146-12-24 19:55:00       NaN   
2  2146-12-31 00:43:00  2147-01-02 17:45:00       NaN   
3  2136-11-20 14:20:00  2136-11-23 14:00:00       NaN   
4  2131-06-04 00:00:00  2131-06-15 16:10:00       NaN   

                admission_type admit_provider_id  admission_location  \
0                     EW EMER.            P278S6      EMERGENCY ROOM   
1                 DIRECT EMER.            P61YAW     CLINIC

In [6]:
# Obtenemos una lista con los pacientes unicos con sepsis

n_pacientes_unicos_sepsis = sepsis_details['subject_id'].unique()
print(n_pacientes_unicos_sepsis)
print(f"Número de pacientes únicos: {sepsis_details['subject_id'].nunique()}")

[10000826 10001401 10001843 ... 19995732 19997367 19997886]
Número de pacientes únicos: 15880


In [7]:
# Filtramos chartevents para quedarnos con los sucesos de los casos con ssepsis_cases

chartevents_sepsis = chartevents[chartevents['subject_id'].isin(n_pacientes_unicos_sepsis)]

# Filtrado por pacientes únicos y múltiples itemid
chartevents_sepsis = chartevents[
    (chartevents['subject_id'].isin(n_pacientes_unicos_sepsis))]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_sepsis.to_csv(ruta_guardado +"chartevents_sepsis_v4.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_sepsis_v4.csv'")

Resultados guardados en 'CSV/chartevents_sepsis_v4.csv'


In [8]:
# Vamos a añadir como características : 
#       1. Nº de los antibiotocos tomados en la ultima hospitalizacion
# Trabajaré sobre la última hospitalización del paciente

chartevents_sepsis = pd.read_csv(ruta_guardado + "chartevents_sepsis_v4.csv")

# Borramos columnas inutiles
chartevents_sepsis = chartevents_sepsis.drop([ 'rate', 'itemid', 'storetime', 'caregiver_id', 'endtime', 'rateuom'], axis=1)

# Convertir charttime con formato explícito para cada grupo
chartevents_sepsis['charttime_numeric'] = pd.to_datetime(chartevents_sepsis['starttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Convertir a segundos desde epoch
chartevents_sepsis['charttime_numeric_s'] = chartevents_sepsis['charttime_numeric'].astype(np.int64) / 10**9
# Obtener la última medición de presión arterial por paciente
ultimo_oxigeno = (
    chartevents_sepsis.groupby('hadm_id')['starttime']
    .max()
    .reset_index()
    .rename(columns={'starttime': 'ultima_charttime'})
)

# Unir con el dataset original para añadir la última hora de medición de cada paciente
df = chartevents_sepsis.merge(ultimo_oxigeno, on='hadm_id', how='left')

df['ultima_charttime'] = pd.to_datetime(df['ultima_charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['ultima_charttime'] = df['ultima_charttime'].astype(np.int64) / 10**9

df = df.drop(['starttime'],axis= 1)

df.to_csv(ruta_guardado +"borrar.csv", index=False)


In [9]:
# print(df.head())

In [10]:
# Elimino columnas innecesarias
df = df.drop(['stay_id', 'orderid', 'linkorderid', 'ordercategoryname', 'secondaryordercategoryname',
              'ordercomponenttypedescription', 'totalamount', 'isopenbag', 'continueinnextdept', 
             'statusdescription', 'originalamount', 'originalamount', 'originalrate', 
              'ordercategorydescription'], axis=1)

print(df.head())

   subject_id   hadm_id  amount amountuom  patientweight totalamountuom  \
0    10002013  23581541     1.0      dose           96.0             mL   
1    10002013  23581541     1.0      dose           96.0             mL   
2    10002428  20321825     1.0      dose           55.0             mL   
3    10002428  20321825     1.0      dose           55.0             mL   
4    10002428  20321825     1.0      dose           55.0             mL   

    charttime_numeric  charttime_numeric_s  ultima_charttime  
0 2160-05-18 22:00:00         6.007817e+09      6.007848e+09  
1 2160-05-19 06:45:00         6.007848e+09      6.007848e+09  
2 2156-05-01 10:00:00         5.880074e+09      5.880172e+09  
3 2156-05-01 13:12:00         5.880086e+09      5.880172e+09  
4 2156-05-02 13:00:00         5.880172e+09      5.880172e+09  


In [11]:
# Paso 1: Identificar la última hospitalización por subject_id
ultima_hospitalizacion = (
    df.groupby('subject_id')['ultima_charttime']
    .max()
    .reset_index()
    .rename(columns={'ultima_charttime': 'max_ultima_charttime'})
)

# Paso 2: Unir esta información con el DataFrame original
df_filtrado = df.merge(ultima_hospitalizacion, on='subject_id', how='left')

# Paso 3: Filtrar solo las filas donde ultima_charttime coincide con la última hospitalización
df_ultima_hosp = df_filtrado[df_filtrado['ultima_charttime'] == df_filtrado['max_ultima_charttime']]

# Paso 4: Opcional: eliminar la columna auxiliar 'max_ultima_charttime' si no es necesaria
df_ultima_hosp = df_ultima_hosp.drop(columns=['max_ultima_charttime', 'hadm_id'])

# Guardar el resultado final
df_ultima_hosp.to_csv(ruta_guardado + "ultima_hospitalizacion.csv", index=False)

In [12]:
print(df_ultima_hosp.head())

   subject_id  amount amountuom  patientweight totalamountuom  \
0    10002013     1.0      dose           96.0             mL   
1    10002013     1.0      dose           96.0             mL   
5    10002428     1.0      dose           48.4             mL   
6    10002428     1.0      dose           48.4             mL   
7    10002428     1.0      dose           48.4             mL   

    charttime_numeric  charttime_numeric_s  ultima_charttime  
0 2160-05-18 22:00:00         6.007817e+09      6.007848e+09  
1 2160-05-19 06:45:00         6.007848e+09      6.007848e+09  
5 2156-05-13 22:11:00         5.881155e+09      5.881752e+09  
6 2156-05-14 00:00:00         5.881162e+09      5.881752e+09  
7 2156-05-14 06:18:00         5.881184e+09      5.881752e+09  


In [13]:
df_conteo = df_ultima_hosp.groupby("subject_id").size().reset_index(name="admin_antibioticos")

print(df_conteo.head())

   subject_id  admin_antibioticos
0    10002013                   2
1    10002428                  33
2    10003019                   3
3    10003400                  24
4    10003637                   5


In [14]:
# Añado esta característica al fichero de características del MODELO2_V3 (parte sepsis)

archivo_estadísticas = pd.read_csv("C:/Users/BiDAlab/TFG_Alfonso/Pruebas/Modelo2/MIMIC/CSV/V3/archivo_combiando_oxigeno_max_min.csv")

archivo_estadísticas_sepsis = pd.merge(archivo_estadísticas, df_conteo, on='subject_id', how='left')
archivo_estadísticas_sepsis["admin_antibioticos"] = archivo_estadísticas_sepsis["admin_antibioticos"].fillna(0)

print(archivo_estadísticas_sepsis.head())

   subject_id  gender  anchor_age  sis_desviacion  dias_desviacion  \
0    10002013       0          53       16.689746         8.576824   
1    10002155       0          80        9.555103         7.056912   
2    10002428       0          80       16.925170        12.605705   
3    10002760       1          56        9.953648         4.998814   
4    10003400       0          72       10.814669         9.766610   

   mean_desviacion   sis_media  dias_media  mean_media  sis_mediana  ...  \
0        13.514515  110.619048   61.476190   76.700000        116.0  ...   
1         8.105554  113.400000   57.400000   80.200000        115.0  ...   
2        15.075749  110.987220   59.006390   79.524845        109.0  ...   
3         5.984499  108.434783   57.521739   74.217391        107.0  ...   
4         9.163913  102.727749   64.884817   78.518325        102.0  ...   

   mean_mediana  dias_tendencia  sis_tendencia  mean_tendencia  sepsis  \
0          75.5   -2.027962e-04  -2.373182e-04  

In [15]:
# Realizamos lo mismo pero para los pacientes sin sepsis

# Ahora hago una lista con los pacientes sin sepsis
subject_ids_sepsis_0 = archivo_estadísticas[archivo_estadísticas['sepsis'] == 0]['subject_id'].tolist()

# Miro en chatevents su oxigeno

# Filtramos chartevents para quedarnos con los sucesos de los casos SIN sepsis
chartevents_no_sepsis = chartevents[chartevents['subject_id'].isin(subject_ids_sepsis_0)]

# Guardar los detalles de sepsis en un archivo CSV
chartevents_no_sepsis.to_csv(ruta_guardado +"chartevents_NO_sepsis_v4.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/chartevents_NO_sepsis_v4.csv'")

Resultados guardados en 'CSV/chartevents_NO_sepsis_v4.csv'


In [16]:
# Vamos a añadir como características : 
#       1. Nº de los antibiotocos tomados en la ultima hospitalizacion
# Trabajaré sobre la última hospitalización del paciente

chartevents_no_sepsis = pd.read_csv(ruta_guardado + "chartevents_NO_sepsis_v4.csv")

# Borramos columnas inutiles
chartevents_no_sepsis = chartevents_no_sepsis.drop([ 'rate', 'itemid', 'storetime', 'caregiver_id', 'endtime', 'rateuom'], axis=1)

# Convertir charttime con formato explícito para cada grupo
chartevents_no_sepsis['charttime_numeric'] = pd.to_datetime(chartevents_no_sepsis['starttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

# Convertir a segundos desde epoch
chartevents_no_sepsis['charttime_numeric_s'] = chartevents_no_sepsis['charttime_numeric'].astype(np.int64) / 10**9
# Obtener la última medición de presión arterial por paciente
ultimo_oxigeno = (
    chartevents_no_sepsis.groupby('hadm_id')['starttime']
    .max()
    .reset_index()
    .rename(columns={'starttime': 'ultima_charttime'})
)

# Unir con el dataset original para añadir la última hora de medición de cada paciente
df = chartevents_no_sepsis.merge(ultimo_oxigeno, on='hadm_id', how='left')

df['ultima_charttime'] = pd.to_datetime(df['ultima_charttime'], format='%Y-%m-%d %H:%M:%S', errors='coerce')

df['ultima_charttime'] = df['ultima_charttime'].astype(np.int64) / 10**9

df = df.drop(['starttime'],axis= 1)

df.to_csv(ruta_guardado +"borrar.csv", index=False)


In [17]:
# Elimino columnas innecesarias
df = df.drop(['stay_id', 'orderid', 'linkorderid', 'ordercategoryname', 'secondaryordercategoryname',
              'ordercomponenttypedescription', 'totalamount', 'isopenbag', 'continueinnextdept', 
             'statusdescription', 'originalamount', 'originalamount', 'originalrate', 
              'ordercategorydescription'], axis=1)

print(df.head())

   subject_id   hadm_id      amount amountuom  patientweight totalamountuom  \
0    10002155  20345487    1.000000      dose           21.1             mL   
1    10002155  23822395  499.999976        mg           53.0            NaN   
2    10002155  23822395    1.000000      dose           53.0             mL   
3    10002155  23822395    1.000000      dose           53.0            NaN   
4    10002155  23822395    1.000000      dose           53.0            NaN   

    charttime_numeric  charttime_numeric_s  ultima_charttime  
0 2131-03-10 02:00:00         5.086548e+09      5.086548e+09  
1 2129-08-07 15:00:00         5.036483e+09      5.036746e+09  
2 2129-08-07 15:00:00         5.036483e+09      5.036746e+09  
3 2129-08-07 15:00:00         5.036483e+09      5.036746e+09  
4 2129-08-08 09:00:00         5.036548e+09      5.036746e+09  


In [18]:
# Paso 1: Identificar la última hospitalización por subject_id
ultima_hospitalizacion = (
    df.groupby('subject_id')['ultima_charttime']
    .max()
    .reset_index()
    .rename(columns={'ultima_charttime': 'max_ultima_charttime'})
)

# Paso 2: Unir esta información con el DataFrame original
df_filtrado = df.merge(ultima_hospitalizacion, on='subject_id', how='left')

# Paso 3: Filtrar solo las filas donde ultima_charttime coincide con la última hospitalización
df_ultima_hosp = df_filtrado[df_filtrado['ultima_charttime'] == df_filtrado['max_ultima_charttime']]

# Paso 4: Opcional: eliminar la columna auxiliar 'max_ultima_charttime' si no es necesaria
df_ultima_hosp = df_ultima_hosp.drop(columns=['max_ultima_charttime', 'hadm_id'])

# Guardar el resultado final
df_ultima_hosp.to_csv(ruta_guardado + "ultima_hospitalizacion_NO_sepsis.csv", index=False)

In [19]:
df_conteo = df_ultima_hosp.groupby("subject_id").size().reset_index(name="admin_antibioticos")

print(df_conteo.head())

   subject_id  admin_antibioticos
0    10002155                   1
1    10002760                   2
2    10004606                   2
3    10004764                   3
4    10005866                  10


In [20]:
# Añado esta característica al fichero de características del MODELO2_V2 (parte ssepsis_cases)

archivo_estadísticas_NO_sepsis = pd.merge(archivo_estadísticas, df_conteo, on='subject_id', how='left')
archivo_estadísticas_NO_sepsis["admin_antibioticos"] = archivo_estadísticas_sepsis["admin_antibioticos"].fillna(0)


print(archivo_estadísticas_NO_sepsis.head())

   subject_id  gender  anchor_age  sis_desviacion  dias_desviacion  \
0    10002013       0          53       16.689746         8.576824   
1    10002155       0          80        9.555103         7.056912   
2    10002428       0          80       16.925170        12.605705   
3    10002760       1          56        9.953648         4.998814   
4    10003400       0          72       10.814669         9.766610   

   mean_desviacion   sis_media  dias_media  mean_media  sis_mediana  ...  \
0        13.514515  110.619048   61.476190   76.700000        116.0  ...   
1         8.105554  113.400000   57.400000   80.200000        115.0  ...   
2        15.075749  110.987220   59.006390   79.524845        109.0  ...   
3         5.984499  108.434783   57.521739   74.217391        107.0  ...   
4         9.163913  102.727749   64.884817   78.518325        102.0  ...   

   mean_mediana  dias_tendencia  sis_tendencia  mean_tendencia  sepsis  \
0          75.5   -2.027962e-04  -2.373182e-04  

In [22]:
# Combino los archivos de sepsis y de no ssepsis_cases

archivo_final = pd.concat([archivo_estadísticas_sepsis, archivo_estadísticas_NO_sepsis])

# Ordenar por subject_id de menor a mayor
archivo_final = archivo_final.sort_values(by='subject_id').reset_index(drop=True)

archivo_final = archivo_final.drop_duplicates(subset=["subject_id"])

# Guardar los detalles en un archivo CSV
archivo_final.to_csv(ruta_guardado +"archivo_combiando_v4.csv", index=False)
# Mensaje de finalización
print("Resultados guardados en 'CSV/archivo_combiando_v4.csv'")

Resultados guardados en 'CSV/archivo_combiando_v4.csv'
